In [1]:
import pandas as pd

In [4]:
losAngeles = pd.read_csv('data.csv', low_memory = False)

In [54]:
losAngeles.head()

,Year,Department Title,Payroll Department,Record Number,Job Class Title,Employment Type,Hourly or Event Rate,Projected Annual Salary,Q1 Payments,Q2 Payments,...,MOU Title,FMS Department,Job Class,Pay Grade,Average Health Cost,Average Dental Cost,Average Basic Life,Average Benefit Cost,Benefits Plan,Job Class Link
Row ID,,,,,,,,,,,,,,,,,,,,,
111391,2014,Water And Power (DWP),NaN,1412316577,Commercial Service Representative,Full Time,NaN,$70386.48,$16129.89,$17829.78,...,NaN,98,1230,NaN,$16842.08,$1282.61,$0.00,$18124.69,DWP,http://per.lacity.org/perspecs/1230.pdf
31732,2013,Police (LAPD),4301.0,432728338,Police Officer I,Full Time,$25.12,$52450.56,$11331.00,$13859.93,...,POLICE OFFICERS UNIT,70,2214,A,$11651.40,$898.08,$191.04,$12740.52,Police,http://per.lacity.org/perspecs/2214.pdf
27697,2013,Police (LAPD),4301.0,97182506,Police Officer II,Full Time,$42.77,$89303.76,$20036.32,$23479.20,...,POLICE OFFICERS UNIT,70,2214,2,$11651.40,$898.08,$191.04,$12740.52,Police,http://per.lacity.org/perspecs/2214.pdf
14136,2013,Harbor (Port of LA),3201.0,950136941,Senior Security Officer,Full Time,$28.75,$60028.96,$15793.88,$18560.38,...,SUPV BLUE COLLAR,42,3184,0,$10710.24,$405.24,$11.40,$11126.88,City,http://per.lacity.org/perspecs/3184.pdf
91896,2014,Public Works - Sanitation,7024.0,3230003445,Senior Clerk Typist,Full Time,$30.92,$64553.13,$14700.00,$17313.10,...,CLERICAL UNIT,82,1368,0,$11000.40,$409.68,$11.40,$11421.48,City,http://per.lacity.org/perspecs/1368.pdf


### Overview of Los Angeles City Payroll Columns

In [7]:
losAngeles.columns

Index(['Row ID', 'Year', 'Department Title', 'Payroll Department',
       'Record Number', 'Job Class Title', 'Employment Type',
       'Hourly or Event Rate', 'Projected Annual Salary', 'Q1 Payments',
       'Q2 Payments', 'Q3 Payments', 'Q4 Payments', 'Payments Over Base Pay',
       '% Over Base Pay', 'Total Payments', 'Base Pay', 'Permanent Bonus Pay',
       'Longevity Bonus Pay', 'Temporary Bonus Pay', 'Lump Sum Pay',
       'Overtime Pay', 'Other Pay & Adjustments',
       'Other Pay (Payroll Explorer)', 'MOU', 'MOU Title', 'FMS Department',
       'Job Class', 'Pay Grade', 'Average Health Cost', 'Average Dental Cost',
       'Average Basic Life', 'Average Benefit Cost', 'Benefits Plan',
       'Job Class Link'],
      dtype='object')

### Unique cells checked in Row ID to determine column's suitability for DataFrame ID.

In [40]:
losAngeles['Row ID'].nunique()

285008

### Take a look at size of the file to determine which fields can be turned into 'category' in order to reference a small list of values

In [8]:
losAngeles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285008 entries, 0 to 285007
Data columns (total 35 columns):
Row ID                          285008 non-null int64
Year                            285008 non-null int64
Department Title                285008 non-null object
Payroll Department              231463 non-null float64
Record Number                   285008 non-null object
Job Class Title                 285008 non-null object
Employment Type                 285008 non-null object
Hourly or Event Rate            263987 non-null object
Projected Annual Salary         285008 non-null object
Q1 Payments                     285008 non-null object
Q2 Payments                     285008 non-null object
Q3 Payments                     285008 non-null object
Q4 Payments                     285008 non-null object
Payments Over Base Pay          285008 non-null object
% Over Base Pay                 285008 non-null object
Total Payments                  285008 non-null object
Base Pay  

### Unique categories counted for likely candidate in order to reduce size.

In [23]:
losAngeles['Employment Type'].nunique()

3

In [38]:
losAngeles['Employment Type'] = losAngeles['Employment Type'].astype('category')
losAngeles['Department Title'] = losAngeles['Department Title'].astype('category')
losAngeles['Year'] = losAngeles['Year'].astype('category')
losAngeles['Payroll Department'] = losAngeles['Payroll Department'].astype('category')
losAngeles['FMS Department'] = losAngeles['FMS Department'].astype('category')
losAngeles['Pay Grade'] = losAngeles['Pay Grade'].astype('category')
losAngeles['Job Class'] = losAngeles['Job Class'].astype('category')
losAngeles['MOU Title'] = losAngeles['MOU Title'].astype('category')
losAngeles['Benefits Plan'] = losAngeles['Benefits Plan'].astype('category')

In [39]:
losAngeles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285008 entries, 0 to 285007
Data columns (total 35 columns):
Row ID                          285008 non-null int64
Year                            285008 non-null category
Department Title                285008 non-null category
Payroll Department              231463 non-null category
Record Number                   285008 non-null object
Job Class Title                 285008 non-null object
Employment Type                 285008 non-null category
Hourly or Event Rate            263987 non-null object
Projected Annual Salary         285008 non-null object
Q1 Payments                     285008 non-null object
Q2 Payments                     285008 non-null object
Q3 Payments                     285008 non-null object
Q4 Payments                     285008 non-null object
Payments Over Base Pay          285008 non-null object
% Over Base Pay                 285008 non-null object
Total Payments                  285008 non-null object
Ba

### Set DataFrame 'Row ID' as index as it is unique

In [44]:
losAngeles.set_index(['Row ID'], inplace = True)

### Group by Year

In [47]:
Grouped_losAngeles = losAngeles.groupby('Year')

In [53]:
Grouped_losAngeles.groups

{2013: Int64Index([31732, 27697, 14136, 53269, 22672, 52209, 55051, 48216, 13424,
             48564,
             ...
              8365,  8366,  8367,  8368,  8369,  8370,  8371,  8375,  8376,
              8377],
            dtype='int64', name='Row ID', length=57379),
 2014: Int64Index([111391,  91896, 106560,  59695,  71037,  69509, 110008, 106631,
             100155,  66296,
             ...
              99990,  99991,  99992,  99993,  99994,  99995,  99996,  99997,
              99998,  99999],
            dtype='int64', name='Row ID', length=56948),
 2015: Int64Index([114364, 114365, 114366, 114367, 114368, 114369, 114370, 114372,
             114373, 117505,
             ...
             174080, 174082, 174083, 174084, 174085, 174086, 174089, 174090,
             174092, 174093],
            dtype='int64', name='Row ID', length=59767),
 2016: Int64Index([228450, 228451, 228452, 228453, 228454, 228455, 228456, 228457,
             228458, 266359,
             ...
            